In [1]:
import pandas as pd
import numpy as np
import cvxpy as cp
from scipy.optimize import minimize
from scipy.optimize import newton
from scipy.optimize import root_scalar
import random
import os
import csv
import time
import warnings
warnings.filterwarnings("ignore")
from utils import *

In [2]:
folder_path = 'data'  # Replace with your folder path
file_names = os.listdir(folder_path)
file_names = [os.path.join('data', f) for f in file_names if os.path.isfile(os.path.join(folder_path, f))]
for path in file_names:
    with open(path, 'r', newline='', encoding="utf-8") as csvfile:
        meta = {}
        projects = {}
        votes = {}
        section = ""
        header = []
        reader = csv.reader(csvfile, delimiter=';')
        for row in reader:
            if str(row[0]).strip().lower() in ["meta", "projects", "votes"]:
                section = str(row[0]).strip().lower()
                header = next(reader)
            elif section == "meta":
                meta[row[0]] = row[1].strip()
            elif section == "projects":
                projects[row[0]] = {}
                for it, key in enumerate(header[1:]):
                    projects[row[0]][key.strip()] = row[it+1].strip()
            elif section == "votes":
                votes[row[0]] = {}
                for it, key in enumerate(header[1:]):
                    votes[row[0]][key.strip()] = row[it+1].strip()
        string_list  = [v['vote'] for v in votes.values()]
        parsed = [list(map(int, s.split(','))) for s in string_list]
        unique_ints = sorted(set(i for row in parsed for i in row))
        int_to_col = {num: idx for idx, num in enumerate(unique_ints)}
        A = np.zeros((len(parsed), len(unique_ints)), dtype=float)
        for i, row in enumerate(parsed):
            for num in row:
                A[i, int_to_col[num]] = 1.0
        project_costs = [int(v['cost']) for v in projects.values()]
        k = int(int(meta['budget'])/(sum(project_costs)/len(project_costs)))
        if k>A.shape[1]:
            k = int(A.shape[1]/2)
        print('============================================')
        print('Working on '+path, 'n='+str(A.shape[0]), 'm='+str(A.shape[1]))
        print('============================================')
        print('Computing the fractional Nash core solution...')
        start_time = time.time()
        w_f, x_f = frac_core(k, A, cap=None)
        print('w=')
        print(w_f)
        print('x=')
        print(x_f)
        end_time = time.time()
        print(f"Execution time for the fractional Nash core solution: {end_time - start_time:.4f} seconds")
        print('============================================')
        print('Computing the (discrete) weak Nash core solution...')
        start_time = time.time()
        w_d, x_d = disc_core(k, A, cap=None, w=w_f, x=x_f)
        print('w=')
        print(w_d)
        print('x=')
        print(x_d)
        end_time = time.time()
        print(f"Execution time for the (discrete) weak Nash core solution: {end_time - start_time:.4f} seconds")
        print('============================================')
        print('.')

Working on data\netherlands_amsterdam_252_.pb n=8520 m=85
Computing the fractional Nash core solution...
current epsilon: 0.5459195623758655
current epsilon: 0.3992856862073447
current epsilon: 0.31405585774909545
current epsilon: 0.25381807352165053
current epsilon: 0.2079240501215979
current epsilon: 0.1718525976778536
current epsilon: 0.14417825836990664
current epsilon: 0.12156207602633672
current epsilon: 0.10317981949752995
current epsilon: 0.08782304988534456
current epsilon: 0.0758086665519313
current epsilon: 0.06539343687449947
current epsilon: 0.0570237315416745
current epsilon: 0.0499917713749527
current epsilon: 0.043871726172306856
current epsilon: 0.03869934261015424
current epsilon: 0.03429525147684139
current epsilon: 0.030537255668734397
current epsilon: 0.027080775294936937
current epsilon: 0.024424764910826798
current epsilon: 0.021821251314554323
current epsilon: 0.019779544136760117
current epsilon: 0.017867596715895315
current epsilon: 0.016295638831040033
curren